En esta parte se utilizará otro conjunto muy conocido de datos para aprendizaje, que consiste también de textos en inglés. Se trata de un conjunto de noticias de prensa de la Agencia Reuters, *Reuters-21578 *, https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html

Las noticias han sido categorizadas a mano, con etiquetas de varios tipos, incluyendo *temas, lugares, organizaciones, personas* y otros criterios. Están en formato SGML, las etiquetas aparecen embebidas en el texto. Existe documentación en el archivo README de la distribución.

Nos interesaremos en el tipo de etiqueta *temas (en inglés en los textos topics)*. El objetivo es nuevamente aprender los temas a partir de los textos de las noticias, con la diferencia de que este es un problema multi-etiqueta : cada noticia puede tener varios temas. En vez de resolver el problema multi-etiqueta inicial (con más de 100 tópicos distintos) les pedimos que lo transformen según las siguientes simplificaciones: considere solo los 3 temas más frecuentes, y transforme el problema multi-etiqueta en 3 problemas de clasificación binaria

No se realiza para esta parte una especificación detallada, sino que les pedimos a Uds. que armen los pasos de una solución y definan el detalle del notebook para esta parte. Mínimamente se espera que procesen la entrada SGML, y encuentren algún modo de enfocar el problema multi-etiqueta en la versión simplificada. Se debe proponer algún modo de tratar el texto, con eventuales mejoras respecto a la vectorización de la parte 1. Se deben aplicar clasificadores y medir su performance, mínimamente con precision, recall y medida-F. Finalmente, se espera una discusión detallada de todo lo realizado y eventuales propuestas de mejora.

Puntos a tener en cuenta del conjunto de datos:
* En Topics dentro de la etiqueta <REUTERS> puede haber:
    * YES: Tiene al menos una categoría(puede existir algún error y no tener ningúna categoria)
    * NO: No tiene categoría(puede existir algún error y tener alguna categoria, supuestamente en esta versión no sucede)
    * BYPASS: No está indexado, no se verificó la categoría
* Hay etiquetas que dividen el conjunto en train y test
* Etiquetas:
    * <TOPICS>, </TOPICS> [ONCE, SAMELINE]: Lista de categorias. Si hay alguna estan delimitadas por <D></D>.
    * <UNKNOWN>, </UNKNOWN> metadatos desconocidos
    * <TEXT>, </TEXT> cuerpo del texto

##1

Se cargan los archivos a memoria, con la librería BeautifulSoup se parsea cada documento como xml dividiendo los documentos por noticias.

In [ ]:
import glob
#Cargo los archivos como lista de strings en documentos
lista_nombres_archivo = glob.glob('reuters21578/*.sgm')
documentos = []
for nombre_archivo in lista_nombres_archivo:
    with open (nombre_archivo, "r") as archivo:
        documento = archivo.read()
        documentos.append(documento)

In [ ]:
from BeautifulSoup import BeautifulSoup

raw_reuter_news = []
for documento in documentos:
    #coloco el contenido de los tags 'reuters' en otra lista: raw_reuter_news
    #esa lista contendra las noticias separadas en formato raw
    soup = BeautifulSoup(documento)
    for raw_reuter in soup('reuters'):
        raw_reuter_news.append(raw_reuter)

In [ ]:
#verifico cantidad
print len(raw_reuter_news)
assert(21578 == len(raw_reuter_news))

##2

Se cuentan la cantidad de noticias por tema y se consideran los temas más frecuentes.

In [ ]:
raw_news_x_topics = {}
for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    for topics_soup in raw_reuter_soup.topics:
        #para cada topic en la lista de topics 
        #agrego la noticia a un diccionario clave topic y valor lista de noticias con ese topic.
        for topic_soup in topics_soup:
            try:
                #ya existe la lista.
                raw_news_x_topics[topic_soup].append(raw_reuter_soup)
            except KeyError:
                #es el primer valor
                raw_news_x_topics[topic_soup] = [raw_reuter_soup]

In [ ]:
#ordeno el diccionario por cantidad de noticias en cada topic.
sorted_topics = sorted(raw_news_x_topics, key=lambda topic: len(raw_news_x_topics[topic]), reverse=True)
#los tres topics mas usados
primeros_topics = sorted_topics[:3]
for topic in primeros_topics:
        print topic,len(raw_news_x_topics[topic])

##3

Se eliminan los metadatos y se separan los conjuntos de entrenamiento y test, para ésto se considera el atributo "LEWISPLIT". Se realizaran experimentos considerando por un lado noticias que tengan contenido más allá de la fecha y el titulo, sobre éstas noticias se experimentará tomando en cuenta sólo el contenido y tomando el contenido unido a la fecha y titulo. Por otra parte se considerarán todas las noticias aún si no tienen texto además de la fecha y titulo.

In [ ]:
#ya no necesito los metadatos
#separo en train y test segun el criterio lewissplit

### PARA CUANDO BODY NO ES VACIO
#solo body
X_train = []
X_test = []
X_unused = []
#info mas completa
#body + fecha + titulo
#incluye fecha y titulo
X_train_plus = []
X_test_plus = []
X_unused_plus = []
#para etiqueta EARN
Y1_train = []
Y1_test = []
Y1_unused = []
#para etiqueta ACQ
Y2_train = []
Y2_test = []
Y2_unused = []
#para etiqueta MONEY-FX
Y3_train = []
Y3_test = []
Y3_unused = []

##### PARA CUANDO BODY ES VACIO
#incluye body o '' y fecha y titulo
X_train_b = []
X_test_b = []
X_unused_b = []
#para etiqueta EARN
Y1_train_b = []
Y1_test_b = []
Y1_unused_b = []
#para etiqueta ACQ
Y2_train_b = []
Y2_test_b = []
Y2_unused_b = []
#para etiqueta MONEY-FX
Y3_train_b = []
Y3_test_b = []
Y3_unused_b = []

#cuento los body vacios
empty_body = 0
empty_body_news = []

for raw_reuter_soup in raw_reuter_news:
    #para cada noticia
    topics_de_la_noticia = []
    for topics_soup in raw_reuter_soup.topics:
        for topic_soup in topics_soup:
            #para cada topic en la lista de topics 
            topics_de_la_noticia.append(topic_soup)
    #veo si tiene las primeras categorias
    es_earn = 1 if u'earn' in topics_de_la_noticia else 0
    es_acq = 1 if u'acq' in topics_de_la_noticia else 0
    es_money_fx = 1 if u'money-fx' in topics_de_la_noticia else 0
    #extraigo contenido
    body = raw_reuter_soup.find('body').text if raw_reuter_soup.find('body') is not None else None
    title = raw_reuter_soup.find('title').text if raw_reuter_soup.find('title') is not None else None
    date = raw_reuter_soup.find('dateline').text if raw_reuter_soup.find('dateline') is not None else None
    ## BODY NO VACIO
    if body is not None: 
        #conjunto entrenamiento
        if raw_reuter_soup['lewissplit'] == 'TRAIN':
            X_train.append(body)
            X_train_plus.append('\n'.join([title,body,date]))
            Y1_train.append(es_earn)
            Y2_train.append(es_acq)
            Y3_train.append(es_money_fx)
        #conjunto test
        elif raw_reuter_soup['lewissplit'] == 'TEST':
            X_test.append(body)
            X_test_plus.append('\n'.join([title,body,date]))
            Y1_test.append(es_earn)
            Y2_test.append(es_acq)
            Y3_test.append(es_money_fx)
        #otro
        else:
            X_unused.append(body)
            X_unused_plus.append('\n'.join([title,body,date]))
            Y1_unused.append(es_earn)
            Y2_unused.append(es_acq)
            Y3_unused.append(es_money_fx)
    ## BODY VACIO
    else: 
        empty_body = empty_body + 1
        empty_body_news.append(raw_reuter_soup)
    ## SIEMPRE
    #conjunto entrenamiento
    if raw_reuter_soup['lewissplit'] == 'TRAIN':
        X_train_b.append('\n'.join(filter(None,[title,date])))
        Y1_train_b.append(es_earn)
        Y2_train_b.append(es_acq)
        Y3_train_b.append(es_money_fx)
    #conjunto test
    elif raw_reuter_soup['lewissplit'] == 'TEST':
        X_test_b.append('\n'.join(filter(None,[title,date])))
        Y1_test_b.append(es_earn)
        Y2_test_b.append(es_acq)
        Y3_test_b.append(es_money_fx)
    #otro
    else:
        X_unused_b.append('\n'.join(filter(None,[title,date])))
        Y1_unused_b.append(es_earn)
        Y2_unused_b.append(es_acq)
        Y3_unused_b.append(es_money_fx)

In [ ]:
#verifico cantidad noticias
news_n_topics = X_train + X_test + X_unused
total = len(news_n_topics) + empty_body
print "Total: ", total
assert(21578 == total)
print
print "### Noticias con BODY: ",len(news_n_topics)
#verifico cantidad earn
print "EARN: ",sum(Y1_train+Y1_test+Y1_unused)
#verifico cantidad acq
print "ACQ: ",sum(Y2_train+Y2_test+Y2_unused)
#verifico cantidad money-fx
print "MONEY-FX", sum(Y3_train+Y3_test+Y3_unused)
len_X_train = len(X_train)
len_X_test = len(X_test)
len_X_unused = len(X_unused)
print
print "Train: ",len_X_train
print "Test: ",len_X_test
print "Unused: ",len_X_unused
assert(len(news_n_topics)==len_X_train+len_X_test+len_X_unused)
assert(len_X_train==len(Y1_train))
assert(len_X_train==len(Y2_train))
assert(len_X_train==len(Y3_train))
assert(len_X_test==len(Y1_test))
assert(len_X_test==len(Y2_test))
assert(len_X_test==len(Y3_test))
print
news_n_topics_b = X_train_b + X_test_b + X_unused_b
print "### Noticias con o sin BODY: ",len(news_n_topics_b)
#verifico cantidad earn
print "EARN: ",sum(Y1_train_b+Y1_test_b+Y1_unused_b)
#verifico cantidad acq
print "ACQ: ",sum(Y2_train_b+Y2_test_b+Y2_unused_b)
#verifico cantidad money-fx
print "MONEY-FX", sum(Y3_train_b+Y3_test_b+Y3_unused_b)
len_X_train_b = len(X_train_b)
len_X_test_b = len(X_test_b)
len_X_unused_b = len(X_unused_b)
print
print "Train: ",len_X_train_b
print "Test: ",len_X_test_b
print "Unused: ",len_X_unused_b

In [ ]:
#Organizacion de experimentos

## NOTICIAS CON BODY
Experimentos_A = [
    ("EARN",X_train, X_test, Y1_train, Y1_test),
    ("ACQ",X_train, X_test, Y2_train, Y2_test),
    ("MONEY-FX",X_train, X_test, Y3_train, Y3_test)
]

## NOTICIAS CON BODY CONSIDERANDO FECHA Y TITULO
Experimentos_B = [
    ("EARN",X_train_plus, X_test_plus, Y1_train, Y1_test),
    ("ACQ",X_train_plus, X_test_plus, Y2_train, Y2_test),
    ("MONEY-FX",X_train_plus, X_test_plus, Y3_train, Y3_test)
]

## NOTICIAS CON O SIN BODY CONSIDERANDO FECHA Y TITULO
Experimentos_C = [
    ("EARN",X_train_b, X_test_b, Y1_train_b, Y1_test_b),
    ("ACQ",X_train_b, X_test_b, Y2_train_b, Y2_test_b),
    ("MONEY-FX",X_train_b, X_test_b, Y3_train_b, Y3_test_b)
]

In [ ]:
#Vectorizacion
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect2 = CountVectorizer(min_df=1)
tidf_vect = TfidfVectorizer()
hash_vect = HashingVectorizer(non_negative=True)
Vectorizadores = [    
    ("TfidfVectorizer", tidf_vect),
    ("CountVectorizer", count_vect),
    ("CountVectorizer2", count_vect2),
    ("HashingVectorizer", hash_vect)
]

In [ ]:
#Comparación de diferentes clasificadores
#Fuente: http://scikit-learn.org/stable/auto_examples/calibration/plot_compare_calibration.html

# Creación de clasificadores
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import MultinomialNB

"""
lr = LogisticRegression()
gnb = GaussianNB()
svc = LinearSVC(C=1.0)
rfc = RandomForestClassifier(n_estimators=100)
"""
multiNB = MultinomialNB()
sgd = SGDClassifier()
perceptron = Perceptron()
passiveaggclass = PassiveAggressiveClassifier()
passiveaggclass2 = PassiveAggressiveClassifier(loss='hinge',C=1.0)
passiveaggclass3 = PassiveAggressiveClassifier(loss='squared_hinge',C=1.0)

Clasificadores = [    
    ("MultinomialNB", multiNB),
    ("SGD", sgd),
    ("Perceptron", perceptron),
    ("SGDClassifier", sgd),
    ("Passive-Aggressive I", passiveaggclass),
    ("Passive-Aggressive II", passiveaggclass2),
    ("Passive-Aggressive III", passiveaggclass3)
]

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
resultados = []

#Funcion que recibe 
#X conj entrenamiento
#Y conj test
#V conj vectorizadores
#C conj clasificadores
def run(train_data,train_target,test_data,test_target,V,C,matrix = True):
    resultado = []
    for nomvect, vectorizador in V:
        #Vectorizo
        X_train = vectorizador.fit_transform(train_data)        
        X_test = vectorizador.transform(test_data)
        f1_list = []
        for nomclf,clasificador in C:
            if (matrix):
                print "F1"
                print "C: " + nomclf + ", V: " + nomvect   
            if nomclf == 'RandomForest':
                X_train = X_train.toarray()
                X_test = X_test.toarray()
            #Entreno
            clasificador.fit(X_train, train_target)
            #Predigo Test
            prediccion = 0
            prediccion = clasificador.predict(X_test)  
            #Resultados
            f1 = metrics.f1_score(test_target, prediccion, average='weighted')
            f1_list.append(f1)
            if (matrix):
                print f1
                print confusion_matrix(test_target,prediccion)
        resultado.append(f1_list)
    noms_vects, e = zip(*V)
    noms_clsfs, e = zip(*C)
    return (noms_vects,noms_clsfs,resultado)

In [ ]:
from matplotlib.pyplot import cm
import numpy as np
import matplotlib.pyplot as plt

def plot(clasificadores, vectorizadores, medidaf):
    
    color=cm.rainbow(np.linspace(0,1,100))
    indices = np.arange(len(clasificadores))

    plt.figure(figsize=(12, 8))
    c = 1
    contador = 0
    i = 0
    for vect in vectorizadores:
        plt.barh(indices + i, medidaf[contador], .2, label=vect, color=color[c])
        c=c+10
        contador = contador + 1
        i = i + .3
    plt.yticks(())
    plt.legend(loc='best')
    plt.subplots_adjust(left=.25)
    plt.subplots_adjust(top=.95)
    plt.subplots_adjust(bottom=.05)

    for i, c in zip(indices, clasificadores):
        plt.text(-1.0, i, c)

    plt.title("Medida-f por metodo y por clasificador.")
    plt.show()

In [ ]:
for nomcat,X_train,X_test,y_train,y_test in Experimentos_A:
    print "###### Categoria: " + nomcat
    print
    noms_vects,nomclf,resultados = run(X_train,y_train,X_test,y_test,Vectorizadores,Clasificadores,False)
    plot(nomclf, noms_vects, resultados)

In [ ]:
for nomcat,X_train,X_test,y_train,y_test in Experimentos_B:
    print "###### Categoria: " + nomcat
    print
    run(X_train,y_train,X_test,y_test,Vectorizadores,Clasificadores,False)

In [ ]:
for nomcat,X_train,X_test,y_train,y_test in Experimentos_C:
    print "###### Categoria: " + nomcat
    print
    run(X_train,y_train,X_test,y_test,Vectorizadores,Clasificadores,False)

In [ ]:
from matplotlib.pyplot import cm
import numpy as np
import matplotlib.pyplot as plt

#clasificadores
clf = ["NB", "KNN", "ID3", "Red Neuronal"]
vectores = ["CountVectorizer", "tf-idf", "HashingVectorizer"]
medidas = [[5, 3, 2, 3], [1, 5, 2, 5], [3, 2, 7, 2]]

plot(clf, vectores, medidas)